In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import torch.nn as nn
import sys
import numpy as np
sys.path.append("../VAE_standard")
from models import DNADataset, ALPHABET, SEQ_LENGTH, LATENT_DIM, VAE

sys.path.append("..")
import utils

import Bio.Data.CodonTable

In [ ]:
"""
tokenizer: 
input_ids - torch.LongTensor of shape (batch_size, sequence_length)
attention_mask - torch.Tensor of shape (batch_size, sequence_length), Mask values selected in {0,1}, where 0 := masked, 1 := not masked
"""

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = AutoModelForMaskedLM.from_pretrained("facebook/esm2_t6_8M_UR50D")

MAX_TOKEN_LENGTH = 1024
print(model)

In [ ]:
x = Bio.Data.CodonTable.standard_dna_table
print(x)

In [ ]:
dataset = DNADataset(f"../data/training_spike.fasta")
sequences = [utils.get_genome(np.dot(x[0], np.arange(len(ALPHABET)))) for x in dataset]
str_sequences = ["".join(seq).replace("-","") for seq in sequences]
codon_sequences = [[x.forward_table.get(seq[3*i:3*i+3], "stop") for i in range(len(seq) // 3 - 1)] for seq in str_sequences]
codon_subsequences = [s[:MAX_TOKEN_LENGTH] for s in codon_sequences]

In [ ]:
masked_codon_subsequences = [[seq[:N] + ["<mask>"] + seq[(N+1):] for N in range(MAX_TOKEN_LENGTH)] for seq in codon_subsequences]

In [ ]:
seq = sequences[0]
codon_seq = 
codon_subseq = codon_seq[:MAX_TOKEN_LENGTH]

print(len(seq))
print(len(codon_seq))
print(len(codon_subseq))

N = 0
masked_codon_subseq = codon_subseq[:N] + ["<mask>"] + codon_subseq[(N+1):]

In [ ]:
test_input = tokenizer("".join(masked_codon_subseq), return_tensors="pt", add_special_tokens=False)
test_label = tokenizer("".join(codon_subseq), return_tensors="pt", add_special_tokens=False)["input_ids"]
test_label = torch.where(test_input["input_ids"] == tokenizer.mask_token_id, test_label, -100)

print(test_input)
print(test_label)


with torch.no_grad():
    test = model(**test_input, labels=test_label)
    print(test)

## Per-token BIF algorithm
<img src="./per_tok_BIF.png" width="1000" height="1000"/>

### Define sequences $\{x_i\}_{i=1}^N$ where $x_i = (x_{i,1},...x_{i,S})$ and $S:=$length of sequence. Then, I fix one amino-acid $x_{i,j}$ and calculate $BIF(*,x_{i,j}$)

In [ ]:
def get_loss_vec():
    